In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import norm, pearsonr
from scipy.integrate import quad
import pickle
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
from bayesee.evaluation import *
from bayesee.perception import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
n_location = 19
n_trial = 10000
local_ecc = np.array([0, *[4,]*6, *[6.9295,]*6, *[8,]*6])

In [ ]:
array_p0 = np.array([0, 0.5])
array_d0 = np.array([3,4.5,6,7.5,9])
array_ed = np.array([7])
# array_d0 = np.array([6])
# array_ed = np.array([1,3,5,7,9])
array_e_p0 = np.array([0, 0.5])
array_e_d0 = np.array([3.9, 6.9])
array_e_ed = np.array([7,1e10])

In [ ]:
for p0 in array_p0:
    for d0 in array_d0:
        for ed in array_ed:
            for e_p0 in array_e_p0:
                for e_d0 in array_e_d0:
                    for e_ed in array_e_ed:
                        stimulus_df = pd.DataFrame()
                        response_df = pd.DataFrame()
                        metadata = {}
                        if e_ed == 1e10:
                            model = f'h1_p0_{p0:.1f}_d0_{d0:.1f}_ed_{ed:.1f}_e_p0_{e_p0:.1f}_e_d0_{e_d0:.1f}_e_ed_inf'.replace('.', 'p')
                        else:
                            model = f'h1_p0_{p0:.1f}_d0_{d0:.1f}_ed_{ed:.1f}_e_p0_{e_p0:.1f}_e_d0_{e_d0:.1f}_e_ed_{e_ed:.1f}'.replace('.', 'p')
                            
                        metadata['p0'] = p0
                        metadata['d0'] = d0
                        metadata['ed'] = ed
                        metadata['e_p0'] = e_p0
                        metadata['e_d0'] = e_d0
                        metadata['e_ed'] = e_ed
                        
                        prior = np.array((p0, *(((1-p0) / (n_location),) * (n_location))))
                        e_prior = np.array((e_p0, *(((1-e_p0) / (n_location),) * (n_location))))
                        
                        if e_prior[0] == 0:
                            with np.errstate(divide = 'ignore'):
                                log_prior_ratio = np.log(e_prior)
                        else:
                            log_prior_ratio = np.log(e_prior / e_prior[0])
                            
                        log_likelihood_ratio = np.zeros_like(prior)

                        model_target = np.random.choice(range(n_location+1), size=n_trial, p=prior)
                        model_response = np.zeros_like(model_target)
                        
                        local_dp = d0 * ed / (local_ecc + ed)
                        e_local_dp = e_d0 * e_ed / (local_ecc + e_ed)
                        
                        for index_trial in range(n_trial):
                            array_standard_normal = np.random.normal(size=(n_location,))
                            
                            log_likelihood_ratio[1:] = e_local_dp**2 * (
                                array_standard_normal / local_dp - 1 / 2
                            )

                            if model_target[index_trial] > 0:
                                log_likelihood_ratio[model_target[index_trial]] += (
                                    e_local_dp[model_target[index_trial] - 1]**2
                                )

                            log_posterior_ratio = log_prior_ratio + log_likelihood_ratio
                            model_response[index_trial] = np.argmax(log_posterior_ratio)
                            
                        stimulus_df['location'] = model_target.round()
                        response_df['response_location'] = model_response.round()
                            
                        file_name = repo_path / f"data/search/covert/heuristic/{model}.pickle"
                        stimulus = {"df": stimulus_df, "metadata": metadata}
                        response = {"df": response_df}

                        with open(file_name, "wb") as f:
                            pickle.dump((stimulus, response), f)